# Dependences

In [ ]:
!pip install --upgrade google-api-python-client
!pip install bitsandbytes
!pip install git+https://github.com/huggingface/transformers.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install tiktoken
!pip install torch
!pip install scipy
!pip install einops # Falcon dependency

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-jfht7cax
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-jfht7cax
  Resolved https://github.com/huggingface/transformers.git to commit f92cc7034a49959b247a46a210b912e56a6f977d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-req-build-tdflwk46
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate.git /tmp/pip-req-build-tdflwk46


In [5]:
!nvidia-smi
!free

Mon Aug 21 10:24:50 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.86.05              Driver Version: 535.86.05    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3070        Off | 00000000:01:00.0 Off |                  N/A |
|  0%   52C    P8              17W / 220W |   4267MiB /  8192MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# Empty model cache
!pip install huggingface_hub["cli"]
!huggingface-cli delete-cache

# Imports

In [1]:
import os
import torch
import time
import datetime
import traceback

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

### Generation Settings

In [12]:
# Does not work.
#model.generation_config.do_sample = False
#model.generation_config.max_new_tokens = 2048
#model.generation_config.temperature = .3
#model.generation_config.top_k = 100

DO_SAMPLE = True
MAX_NEW_TOKENS = 2048
TEMPERATURE = .3
TOP_K = 100

# Inference

---
---
---

### >>> XGen 7B Instruct

In [1]:
model_name = 'Salesforce/xgen-7b-8k-inst'

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

NameError: name 'BitsAndBytesConfig' is not defined

In [6]:
PROMPT_TEMPLATE = """A chat between a curious human and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the human's questions.

### Human: Continue the summary of the first part of a dialogue, based on the second part of the same dialogue.

Summary:
{summary}

Second part:
{text}

### Assistant:"""

INTERMEDIATE_FOLDER = 'xgen_7b_inst'

-- Found 30 samples: ['sample_10_9852.txt', 'sample_11_10498.txt', 'sample_12_11166.txt', 'sample_13_4562.txt', 'sample_14_6858.txt', 'sample_15_9989.txt', 'sample_16_6057.txt', 'sample_17_4135.txt', 'sample_18_9825.txt', 'sample_19_9648.txt', 'sample_1_4693.txt', 'sample_20_13365.txt', 'sample_21_7283.txt', 'sample_22_6522.txt', 'sample_23_9169.txt', 'sample_24_10638.txt', 'sample_25_2751.txt', 'sample_26_7860.txt', 'sample_27_9004.txt', 'sample_28_5705.txt', 'sample_29_3855.txt', 'sample_2_11567.txt', 'sample_30_8361.txt', 'sample_3_12019.txt', 'sample_4_13389.txt', 'sample_5_1343.txt', 'sample_6_10232.txt', 'sample_7_11826.txt', 'sample_8_7292.txt', 'sample_9_4578.txt']


In [ ]:
def result_from_output_sequence(output):
    
    # Un-prompt output
    delimiter = '### Assistant:'
    output = delimiter.join(output.split(delimiter)[1:])
    
    # Remove white spaces in front of summary
    while len(output) > 0 and output[0] == ' ':
        output = output[1:]
    
    # Remove <|endoftext|>
    eos_token = '\n<|endoftext|>'
    output = output[:-len(eos_token)]
    
    return output

def generate_output(prompt, counter, chunks):
    input_ids = tokenizer(prompt, return_tensors="pt").to('cuda')
    print('--- Computing chunk', counter, 'of', len(chunks), 'total, size:', len(input_ids['input_ids'][0]), 'tokens')
    output_ids = model.generate(**input_ids, do_sample=DO_SAMPLE, max_new_tokens=MAX_NEW_TOKENS, temperature=TEMPERATURE, eos_token_id=50256, top_k=TOP_K)
    return tokenizer.decode(output_ids[0]).strip()

### >>> Falcon 7B Instruct

In [2]:
model_name = 'tiiuae/falcon-7b-instruct'

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/linagora/.local/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda113.so
CUDA SETUP: CUDA runtime path found: /home/linagora/anaconda3/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 113
CUDA SETUP: Loading binary /home/linagora/.local/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda113.so...


/home/linagora/.local/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/home/linagora/anaconda3/lib/libcudart.so.11.0'), PosixPath('/home/linagora/anaconda3/lib/libcudart.so')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
PROMPT_TEMPLATE = """A chat between a curious human and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the human's questions.
>>QUESTION<<Summarize the following text.

{text}
>>ANSWER<<"""

INTERMEDIATE_FOLDER = 'falcon_7b_inst'

In [8]:
def result_from_output_sequence(output):
    
    # Un-prompt output
    delimiter = '>>ANSWER<<'
    output = delimiter.join(output.split(delimiter)[1:])
    
    # Remove white spaces in front of summary
    while len(output) > 0 and output[0] == ' ':
        output = output[1:]
    
    # Remove <|endoftext|>
    eos_token = '<|endoftext|>'
    output = output[:-len(eos_token)]
    
    return output

def generate_output(prompt, counter, chunks):
    input_ids = tokenizer(prompt, return_token_type_ids=False, return_tensors="pt").to('cuda')
    print('--- Computing chunk', counter, 'of', len(chunks), 'total, size:', len(input_ids['input_ids'][0]), 'tokens')
    output_ids = model.generate(**input_ids, do_sample=DO_SAMPLE, max_new_tokens=MAX_NEW_TOKENS, temperature=TEMPERATURE, top_k=TOP_K)
    return tokenizer.decode(output_ids[0]).strip()

---
---
---

### Loading samples

In [5]:
def mkdir(folder_path):
    try:
        os.mkdir(folder_path)
    except FileExistsError:
        pass

# Load samples from dataset
samples = os.listdir('input/texts')
samples = [sample for sample in samples if sample.endswith('.txt')]
samples.sort()
n_samples = len(samples)
print('-- Found', n_samples, 'samples:', samples)

-- Found 30 samples: ['sample_10_9852.txt', 'sample_11_10498.txt', 'sample_12_11166.txt', 'sample_13_4562.txt', 'sample_14_6858.txt', 'sample_15_9989.txt', 'sample_16_6057.txt', 'sample_17_4135.txt', 'sample_18_9825.txt', 'sample_19_9648.txt', 'sample_1_4693.txt', 'sample_20_13365.txt', 'sample_21_7283.txt', 'sample_22_6522.txt', 'sample_23_9169.txt', 'sample_24_10638.txt', 'sample_25_2751.txt', 'sample_26_7860.txt', 'sample_27_9004.txt', 'sample_28_5705.txt', 'sample_29_3855.txt', 'sample_2_11567.txt', 'sample_30_8361.txt', 'sample_3_12019.txt', 'sample_4_13389.txt', 'sample_5_1343.txt', 'sample_6_10232.txt', 'sample_7_11826.txt', 'sample_8_7292.txt', 'sample_9_4578.txt']


### Chunking algorithm

In [6]:
MAX_CHARACTER_CHUNK_SIZE = 2000 * 3

def chunkize(text):
    """
    Greedy implementation of a dialogue transcript chunking algorithm. This method returns a list of transcript chunks.
    - It priorities stability over performance. There is a set maximum chunk size for LLM inference stability. Really long utterances bypass this limit.
    - It guarantees the cuts are made at utterance ends.

    !! WARNING !! The maximum size is in characters, not tokens. A good upper bound shall be used as there are
    no clear and easy correspondance between characters and tokens, as it obviously depends on the model tokenizer.

    Dialogue format recognized (used in the HF AMI dataset):
    ----------------------
    <A> Hello!
    <B> Hey there
    <A> What have you been up to
    <B> Not much honestly
    ----------------------
    """
    
    chunks = [] # Final list of transcript chunks. This makes up the loop invariant
    utterances = text.split('\n') # Transcript is split into sentences
    utterances.reverse() # Reverse everything!!
    current_chunk = ''

    # While there is still an utterance to process
    while len(utterances) > 0:
        utterance = utterances.pop()

        # Add to current chunk and proceed to next
        if len(current_chunk) + len(utterance) <= MAX_CHARACTER_CHUNK_SIZE:
            if len(current_chunk) > 0:
                current_chunk += '\n'
            current_chunk += utterance
        
        # Utterance is larger than maximum chunk size
        elif len(current_chunk) == 0:
            chunks.append(current_chunk)
            chunks.append(utterance)
            current_chunk = ''

        # Current chunk is big enough, append to list and create new one
        else:
            chunks.append(current_chunk)
            current_chunk = utterance

    if len(current_chunk) > 0:
        chunks.append(current_chunk)

    return chunks

### Merging algorithm

In [7]:
def inference_sample(sample):
    
    # Current summary is the concatenation of all the chunk sub-summaries 
    current_summary = ''
    chunks = chunkize(sample)

    # Split into chunks
    counter = 0
    for chunk in chunks:
        counter += 1

        # Create prompt for this chunk
        prompt = PROMPT_TEMPLATE.format(summary=current_summary, text=chunk)

        # Sample one sub-input
        output = generate_output(prompt, counter, chunks)
        subsummary = result_from_output_sequence(output)

        # Add to summary
        if len(current_summary) > 0:
            current_summary += '\n\n'
        current_summary += subsummary
    
    return current_summary

In [12]:
# Tests for chunking algorithm
import os

sample_file_path = 'input/texts/' + samples[0]
sample_file = open(sample_file_path, 'r', encoding='utf-8')
sample = sample_file.read()
sample_file.close()

chunks = chunkize(sample)
for chunk in chunks:
    print('-----------------------------------------')
    print(chunk)

-----------------------------------------
<UI> All hooked up.
<UI> Okay, so now we are here at the functional design meeting.
<UI> Um hopefully this meeting I'll be doing a little bit less talking than I did last time 'cause this is when you get to show us what you've been doing individually.
<UI> The agenda for the meeting, I put it in the sh shared documents folder.
<UI> I don't know if that meant that you could see it or not.
<UI> Did anyone?
<ME> No.
<ID> Mm.
<UI> No.
<UI> Oh well.
<UI> Um I'll try and do that for the next meeting as well so if you check in there, there's a shared project documents folder.
<UI> Um and it should be in there.
<ME> Mm.
<ME> Um um wi on on a what?
<ME> Oh project project documents, yeah, yeah, yeah, okay.
<UI> Project documents, yeah.
<UI> So I'll put it in there.
<ME> Oh okay, yeah.
<UI> Is it best if I send you an email maybe, to let you know it's there?
<ME> Yes, I think so.
<UI> Yep.
<UI> I'll do that next time.
<UI> Um I'll act as secretary for th

# Computation

In [ ]:
initial_time = time.time()
skipped_samples = 0

mkdir('intermediate')
mkdir('intermediate/' + INTERMEDIATE_FOLDER)

print('Starting computation...')

# For each sample in dataset
for sample_n in range(n_samples):

    # Estimate completion and time.
    cur_samples = sample_n - skipped_samples
    tot_samples = n_samples - skipped_samples
    progress = cur_samples / tot_samples
    pct = round(progress * 100, 1)
    print('Prompting on sample N' + str(sample_n + 1) + '/' + str(n_samples))
    print('-- Completion: ' + str(pct) + '%')
    if cur_samples > 0:
        approx_total = (time.time() - initial_time) / cur_samples * tot_samples
        approx_remaining = approx_total * (1 - progress)
        print('-- Estimated Remaining Time: ' + str(datetime.timedelta(seconds=int(approx_remaining))) + ' (total ' + str(datetime.timedelta(seconds=int(approx_total))) + ')')
    
    # Read sample and generate prompt
    sample_file_path = 'input/texts/' + samples[sample_n]
    sample_file = open(sample_file_path, 'r', encoding='utf-8')
    sample = sample_file.read()
    sample_file.close()
    
    # Find target file
    target_file_path = 'intermediate/' + INTERMEDIATE_FOLDER + '/' + samples[sample_n]
    if os.path.isfile(target_file_path):
        print('-- Found intermediate result file \'' + target_file_path + '\', skipped.')
        skipped_samples += 1
        continue

    try:

        # Inference
        output = inference_sample(sample)

        # Save answer in file
        target_file = open(target_file_path, 'w', encoding='utf-8')
        target_file.write(output)
        target_file.close()
    
    except:
        print('Could not compute prompt:')
        print(sample)
        traceback.print_exc()

delta = time.time() - initial_time
print('Done! Took', datetime.timedelta(seconds=int(delta)), 'seconds')

/home/linagora/anaconda3/lib/python3.10/site-packages/transformers/generation/utils.py:1411: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Starting computation...
Prompting on sample N1/30
-- Completion: 0.0%
-- Found intermediate result file 'intermediate/falcon_7b_inst/sample_10_9852.txt', skipped.
Prompting on sample N2/30
-- Completion: 0.0%
--- Computing chunk 1 of 7 total, size: 1775 tokens


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


--- Computing chunk 2 of 7 total, size: 1696 tokens


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


...

Could not compute prompt:
<ME> Hmm.
<ID> Okay.
<ID> Good morning everybody.
<ID> Um I'm glad you could all come.
<ID> I'm really excited to start this team.
<ID> Um I'm just gonna have a little PowerPoint presentation for us, for our kick-off meeting.
<ID> My name is Rose Lindgren.
<ID> I I'll be the Project Manager.
<ID> Um our agenda today is we are gonna do a little opening and then I'm gonna talk a little bit about the project, then we'll move into acquaintance such as getting to know each other a little bit, including a tool training exercise.
<ID> And then we'll move into the project plan, do a little discussion and close, since we only have twenty five minutes.
<ID> First of all our project aim.
<ID> Um we are creating a new remote control which we have three goals about, it needs to be original, trendy and user-friendly.
<ID> I'm hoping that we can all work together to achieve all three of those.
<ID> Um so we're gonna divide us up into three compa three parts.
<ID> First the f

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


--- Computing chunk 2 of 6 total, size: 1723 tokens


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


--- Computing chunk 3 of 6 total, size: 1771 tokens


# Calculs de scores

In [1]:
!pip install rouge_score rouge
!pip install evaluate
!pip install bert-score
!pip install sacrebleu
!pip install nltk

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 0.0/126.5 kB ? eta -:--:--
     -------------------------------------- 126.5/126.5 kB 7.8 MB/s eta 0:00:00
     ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
     ----------------------------- ---------- 1.1/1.5 MB 34.2 MB/s eta 0:00:01
     ---------------------------------------- 1.5/1.5 MB 31.9 MB/s eta 0:00:00
  Obtaining dependency information for click from https://files.pythonhosted.org/packages/1a/70/e63223f8116931d365993d4a6b7ef653a4d920b41d03de7c59499962821f/click-8.1.6-py3-none-any.whl.metadata
  Obtaining dependency information for joblib from https://files.pythonhosted.org/packages/10

In [4]:
import evaluate
import os

In [7]:
# Methods and variables
print('Starting scores computation...')
bleu = evaluate.load("bleu")
rouge = evaluate.load('rouge')
bertscore = evaluate.load('bertscore')

STORAGE_FILE_NAME = 'scores'

# Find output file for CSV scores
mkdir('output')
storage_file = open('output/' + STORAGE_FILE_NAME + '.csv', 'w', encoding='utf-8')
storage_file.write('path;rouge2;rougel;bertscore\n')

target_file_paths = []
references = []
predictions = []

# For each sample in dataset
for sample_n in range(n_samples):

    # Find generated summary
    target_file_path = 'intermediate/' + INTERMEDIATE_FOLDER + samples[sample_n]
    if not os.path.isfile(target_file_path):
        print('-- Found no intermediate result file \'' + target_file_path + '\', skipped.')
        continue

    # Read sample and generate prompt -> Keep summary
    summary_file_path = 'input/summaries/' + samples[sample_n]
    summary_file = open(summary_file_path, 'r', encoding='utf-8')
    references.append(summary_file.read())
    summary_file.close()

    # Access generated summary
    target_file = open(target_file_path, 'r', encoding='utf-8')
    prediction = target_file.read()
    target_file.close()

    # Add prediction
    target_file_paths.append(target_file_path)
    predictions.append(prediction)

# Calculate metrics
result_rouge = rouge.compute(predictions=predictions, references=references, use_aggregator=False)
result_bertscore = bertscore.compute(predictions=predictions, references=references, lang='fr', rescale_with_baseline=True, verbose=True)

# Write to csv
# Forget about BLEU...
# Format: PATH | ROUGE2 | ROUGEL | BERTScore
for i in range(len(target_file_paths)):
    ligne = target_file_paths[i]
    ligne += ';' + str(result_rouge['rouge2'][i]) + ";" + str(result_rouge['rougeL'][i])
    #ligne += ";" + str(result_bleu['bleu'])
    ligne += ";" + str(result_bertscore['f1'][i])

    storage_file.write(ligne + '\n')

storage_file.close()
print('Done!')

Starting scores computation...
